In [1]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import sys
from datetime import datetime

sys.path.append("../src/")
from mcr_py.package import storage
from mcr_py.package.geometa import GeoMeta
from mcr_py.package.logger import rlog, setup
from mcr_py.package.mcr5.mcr5 import MCR5

setup("INFO")

In [2]:
city_id = "cologne"  # 'Koeln'
city_id_osm = "Koeln"
date = "20240422"
geo_meta_path = f"../data/stateful_variables/{city_id.lower()}_geometa.pkl"
stops = f"../data/gtfs-cleaned/{city_id.lower()}_{date}/stops.csv"
structs = f"../data/gtfs-cleaned/{city_id.lower()}_{date}/structs.pkl"
location_mapping_path = f"../data/city_data/location_mappings_{city_id.lower()}.pkl"
bicycle_base_path = f"../data/sharing_locations_clustered/{city_id.lower()}_bikes/"
mcr5_output_path = f"../data/mcr5/{city_id.lower()}_{date}"
# bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [3]:
geo_meta = GeoMeta.load(geo_meta_path)

In [4]:
lm_data = storage.read_any_dict(location_mapping_path)
location_mappings = lm_data["location_mappings"]
car_location_mappings = lm_data["car_location_mappings"]

In [5]:
from command.step_config import (
    get_bicycle_only_config,
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_public_transport_only_config,
    get_walking_only_config,
)

In [12]:
configs = {}


def get_bicyle_public_transport_config_ready(bicycle_location_path, start_time):
    initial_steps, repeating_steps = get_bicycle_public_transport_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id_osm,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_car_only_config_ready():
    initial_steps, repeating_steps = get_car_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id_osm,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": car_location_mappings,
        "max_transfers": 1,
    }


def get_bicycle_only_config_ready(bicycle_location_path):
    initial_steps, repeating_steps = get_bicycle_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id_osm,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_public_transport_only_config_ready(start_time):
    initial_steps, repeating_steps = get_public_transport_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id_osm,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_walking_only_config_ready():
    initial_steps, repeating_steps = get_walking_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id_osm,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 0,
    }

In [13]:
import itertools

times = [
    "08:00:00",
    "12:00:00",
    "18:00:00",
]
bicycle_location_paths = [
    os.path.join(bicycle_base_path, path) for path in os.listdir(bicycle_base_path)
]

bicycle_public_transport_config_args = list(
    itertools.product(
        times,
        bicycle_location_paths,
    )
)

In [14]:
from functools import partial

for i, (time, bicycle_location_path) in enumerate(bicycle_public_transport_config_args):
    print(i, time, bicycle_location_path)
    configs[f"bicycle_public_transport_{i}"] = partial(
        get_bicyle_public_transport_config_ready, bicycle_location_path, time
    )

print()
for i, time in enumerate(times):
    print(i, time)
    configs[f"public_transport_{i}"] = partial(
        get_public_transport_only_config_ready, time
    )

print()
for i, bicycle_location_path in enumerate(bicycle_location_paths):
    print(i, bicycle_location_path)
    configs[f"bicycle_{i}"] = partial(
        get_bicycle_only_config_ready, bicycle_location_path
    )

configs["car"] = get_car_only_config_ready
configs["walking"] = get_walking_only_config_ready

0 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-11-25_13_01_00.csv
1 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-04-07_18_01_00.csv
2 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2023-02-26_15_01_00.csv
3 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2023-02-24_00_01_00.csv
4 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-11-07_04_01_00.csv
5 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-08-29_17_01_00.csv
6 08:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-12-30_08_01_00.csv
7 12:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-11-25_13_01_00.csv
8 12:00:00 ../data/sharing_locations_clustered/cologne_bikes/2022-04-07_18_01_00.csv
9 12:00:00 ../data/sharing_locations_clustered/cologne_bikes/2023-02-26_15_01_00.csv
10 12:00:00 ../data/sharing_locations_clustered/cologne_bikes/2023-02-24_00_01_00.csv
11 12:00:00 ../data/sharing_locations_clustered/cologne_bikes/20

In [15]:
if os.path.exists(mcr5_output_path):
    raise Exception("Output path already exists")

In [16]:
configs

{'bicycle_public_transport_0': functools.partial(<function get_bicyle_public_transport_config_ready at 0x75487e2a3920>, '../data/sharing_locations_clustered/cologne_bikes/2022-11-25_13_01_00.csv', '08:00:00'),
 'bicycle_public_transport_1': functools.partial(<function get_bicyle_public_transport_config_ready at 0x75487e2a3920>, '../data/sharing_locations_clustered/cologne_bikes/2022-04-07_18_01_00.csv', '08:00:00'),
 'bicycle_public_transport_2': functools.partial(<function get_bicyle_public_transport_config_ready at 0x75487e2a3920>, '../data/sharing_locations_clustered/cologne_bikes/2023-02-26_15_01_00.csv', '08:00:00'),
 'bicycle_public_transport_3': functools.partial(<function get_bicyle_public_transport_config_ready at 0x75487e2a3920>, '../data/sharing_locations_clustered/cologne_bikes/2023-02-24_00_01_00.csv', '08:00:00'),
 'bicycle_public_transport_4': functools.partial(<function get_bicyle_public_transport_config_ready at 0x75487e2a3920>, '../data/sharing_locations_clustered/col

In [17]:
runtimes = {}
for key, config in configs.items():
    start = datetime.now()
    rlog.info(f"Running MCR5 for {key}")

    config = config()
    mcr5 = MCR5(**config["init_kwargs"], max_processes=8)

    loaded_at = datetime.now()
    load_time = loaded_at - start

    output_path = os.path.join(mcr5_output_path, key)

    location_mappings = config["location_mappings"]

    start_time = config.get("start_time", "08:00:00")
    errors = mcr5.run(
        location_mappings,
        start_time=start_time,
        output_dir=output_path,
        max_transfers=config["max_transfers"],
    )

    run_time = datetime.now() - loaded_at
    total_time = datetime.now() - start
    runtimes[key] = {
        "load_time": load_time,
        "run_time": run_time,
        "total_time": total_time,
    }

with open(os.path.join(mcr5_output_path, "runtimes.pkl"), "wb") as f:
    pickle.dump(runtimes, f)

[11:33:02] INFO     Running MCR5 for bicycle_public_transport_0                                      ]8;id=313397;file:///tmp/ipykernel_1226015/542061263.py\542061263.py]8;;\:]8;id=997467;file:///tmp/ipykernel_1226015/542061263.py#4\4]8;;\

           INFO     Downloading OSM data                                                                 ]8;id=686502;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=23066;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#102\102]8;;\

Downloaded Protobuf data 'Koeln.osm.pbf' (77.26 MB) to:
'/tmp/pyrosm/Koeln.osm.pbf'


[11:33:10] INFO     Downloading OSM data done (7.65 seconds)                                             ]8;id=718681;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=589043;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#102\102]8;;\

           INFO     Reading OSM network (walking)                                                         ]8;id=459137;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=196049;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#73\73]8;;\

...

           INFO     Fetching POI for runtime optimization done (0.06 seconds)                     ]8;id=940308;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=18387;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py#51\51]8;;\

[12:47:43] INFO     Reading stops                                                           ]8;id=489490;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=66160;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#143\143]8;;\

           INFO     Reading stops done (0.21 seconds)                                       ]8;id=331809;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=202205;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#143\143]8;;\

Starting:   0%|          | 0/1782 [00:00<?, ?it/s]

Exception ignored in: <function _releaseLock at 0x754adb201260>
Traceback (most recent call last):
  File "/home/ppeter/miniconda3/envs/mcr-py/lib/python3.11/logging/__init__.py", line 237, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


In [ ]:
print("Ready")

In [ ]:
# errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle_public_transport/errors.pkl")
# errors